In [2]:
import tensorflow as tf
import numpy as np

sentence = (
    'Lorem ipsum dolor sit amet consectetur adipisicing elit '
    'sed do eiusmod tempor incididunt ut labore et dolore magna '
    'aliqua Ut enim ad minim veniam quis nostrud exercitation'
)

In [5]:
word_list = list(set(sentence.split()))

word_dict = {w:i for i,w in enumerate(word_list)}
number_dict = {i:w for i,w in enumerate(word_list)}

n_class = len(word_dict)
n_step = len(sentence.split())

n_hidden = 5

def make_batch(sentence):
    input_batch, target_batch = [],[]
    words = sentence.split()
    for i, word in enumerate(words[:-1]):
        input = [word_dict[n] for n in words[:(i+1)]]
        input = input+[0]*(n_step-len(input))
        target = word_dict[words[i+1]]
        input_batch.append(np.eye(n_class)[input])
        target_batch.append(np.eye(n_class)[target])
    return input_batch, target_batch

In [11]:
#bi-lstm model
X = tf.placeholder(tf.float32, shape=[None, n_step, n_class])
Y = tf.placeholder(tf.float32, shape=[None, n_class])

W = tf.Variable(tf.random_uniform([n_hidden*2, n_class]))
b = tf.Variable(tf.random_uniform([n_class]))

lstm_fw_cell = tf.nn.rnn_cell.LSTMCell(n_hidden)
lstm_bw_cell = tf.nn.rnn_cell.LSTMCell(n_hidden)

outputs,_ = tf.nn.bidirectional_dynamic_rnn(lstm_fw_cell, lstm_bw_cell, X, dtype=tf.float32)
#outputs [batch_size, len_seq, n_hidden], states [batch_size, n_hidden]

outputs = tf.concat([outputs[0], outputs[1]], 2)
outputs = tf.transpose(outputs, [1,0,2]) #[n_step, batch_size, n_hidden]
outputs = outputs[-1]

model = tf.matmul(outputs, W)+b

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=model, labels=Y))

optimizer = tf.train.AdamOptimizer(0.001).minimize(cost)

prediction = tf.cast(tf.argmax(model,1), tf.int32)

In [21]:
#training
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

In [27]:
input_batch , target_batch = make_batch(sentence)

In [36]:
for epoch in range(10000):
    _, loss = sess.run([optimizer, cost], feed_dict={X:input_batch,Y:target_batch})
    if (epoch+1) % 1000 ==0:
        print("Epoch : {:5d}, cost={:.5f}".format(epoch+1, loss))

Epoch :  1000, cost=0.33386
Epoch :  2000, cost=0.30247
Epoch :  3000, cost=0.29328
Epoch :  4000, cost=0.28633
Epoch :  5000, cost=0.28096
Epoch :  6000, cost=0.19471
Epoch :  7000, cost=0.15730
Epoch :  8000, cost=0.12521
Epoch :  9000, cost=0.09709
Epoch : 10000, cost=0.07622


In [37]:
predict = sess.run([prediction], feed_dict={X: input_batch})
print(sentence)
print([number_dict[n] for n in [pre for pre in predict[0]]])

Lorem ipsum dolor sit amet consectetur adipisicing elit sed do eiusmod tempor incididunt ut labore et dolore magna aliqua Ut enim ad minim veniam quis nostrud exercitation
['ipsum', 'dolor', 'sit', 'amet', 'consectetur', 'adipisicing', 'elit', 'sed', 'do', 'eiusmod', 'tempor', 'incididunt', 'ut', 'labore', 'et', 'dolore', 'magna', 'aliqua', 'Ut', 'enim', 'minim', 'minim', 'veniam', 'quis', 'nostrud', 'exercitation']
